In [ ]:
import torch
import pickle
import numpy as np

data_name = 'cifar10'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

save_result_path = "./result/{}".format(data_name)



In [ ]:
estimator_list = []
count = 0

skip = 'quad'

for image_index in range(10000):

    num_ddim_steps_1 = 50
    
    image_path = '{}/S_{}/image_{:d}'.format(save_result_path, num_ddim_steps_1, image_index)
    if skip == 'linear':
        noise_filename1 = '{}/model_{}_intermediates.pt'.format(image_path, model_name_1)
        jac_filename1 = '{}/model_{}_jac_logdet.pt'.format(image_path, model_name_1)
    else:
        noise_filename1 = '{}/model_{}_{}_intermediates.pt'.format(image_path, model_name_1, skip)
        jac_filename1 = '{}/model_{}_{}_jac_logdet.pt'.format(image_path, model_name_1, skip)

    num_ddim_steps_2 = 100
    image_path = '{}/S_{}/image_{:d}'.format(save_result_path, num_ddim_steps_2, image_index)
    if skip == 'linear':
        noise_filename2 = '{}/model_{}_intermediates.pt'.format(image_path, model_name_1)
        jac_filename2 = '{}/model_{}_jac_logdet.pt'.format(image_path, model_name_1)
    else:
        noise_filename2 = '{}/model_{}_{}_intermediates.pt'.format(image_path, model_name_1, skip)
        jac_filename2 = '{}/model_{}_{}_jac_logdet.pt'.format(image_path, model_name_1, skip)

    # load saved results to compute the estimator
    if os.path.exists(noise_filename1) and os.path.exists(noise_filename2): 
        f = open(noise_filename1, 'rb')
        intermediates1 = pickle.load(f)
        f.close()
        f = open(jac_filename1, 'rb')
        jac_logdet_model1 = pickle.load(f)
        f.close()
        
        f = open(noise_filename2, 'rb')
        intermediates2 = pickle.load(f)
        f.close()
        f = open(jac_filename2, 'rb')
        jac_logdet_model2 = pickle.load(f)
        f.close()
        
        estimator = 0.5 * intermediates2[-1].pow(2).sum() - 0.5 * intermediates1[-1].pow(2).sum() + jac_logdet_model1 - jac_logdet_model2
        estimator_list.append(estimator.item())



In [ ]:
estimator_list = np.array(estimator_list)
estimator = estimator_list.mean()
CI_lower_bound = estimator_list.mean() - z_half_alpha * estimator_list.std() / np.sqrt(len(estimator_list))
CI_upper_bound = estimator_list.mean() + z_half_alpha * estimator_list.std() / np.sqrt(len(estimator_list))
